# Ngày 3: Transformations & Actions

#### Transformation & action là hai operation trên RDD. Ta cùng tìm hiểu thôi nào :))

### 1. Lazy Evaluation (lazy là lười biếng)

Việc xử lý dữ liệu, nhìn rộng ra, chính là việc biến đổi từ tập dữ liệu này sang tập dữ liêu khác, hay với Spark, là biến đổi từ RDD này sang RDD khác.

**Khái niệm**: Spark không thực hiện tính toán ngay lập tức khi bạn gọi các transformations. Thay vào đó, Spark xây dựng một "DAG" (Directed Acyclic Graph) và chỉ thực hiện tính toán khi có một action được gọi.

- Lợi ích: Tối ưu hóa các bước tính toán
- Giảm thiểu số lượng bước thừa

**Example**

Ví dụ: Ở đây ta có 1 tập dữ liệu với 1 tỉ record chứa dữ liệu về các nước... Ta sẽ thực hiện các
transformation trên tập này như hình bên dưới.

![image](image_1/lazy.png)

**Giải thích**: Ở hình trên ta đang thực hiện 3 transformation là:
- RDD1: Load to memory
- RDD2: group by()
- RDD3: Filter from RDD2
- Action: show()

<p style = "color: green;">Vậy Spark lazy như thế nào ? </p>

Ở đây ta có thể thấy mặc dù đã tạo ra 3 RDD nhưng thực chất không có RDD nào được thực thi cả. Spark cứ
đợi cho đến khi nào có một action được gọi thì spark mới bắt đầu thực hiện phân tích các transfomation và
đưa ra strategy tối ưu nhất.

--> Chính việc đợi và đợi cho đến khi có một action được gọi chính là cơ chế LAZY

<p style = "color: green;">Vậy tại sao Spark lại phải lazy ? </p>

![whylazy](image_1/notlazy.png)


**Giải thích:** Bạn so sánh khi lazy và không lazy xem. 
- Nếu không lazy thì cứ làm việc tuần tự như thế,
nó sẽ load 1 tỷ recore lên memory, tốn rất nhiều bộ nhớ.
- Còn lazy, nó làm biếng nhưng thông minh hơn, nó xem xét thấy bước
filter làm trước thì dữ liệu load lên memory chỉ 5 record,
tiết khiệm rất nhiều dung lượng bộ nhớ.

**Tóm lại:** Việc lazy là để cho spark có khả năng tính toán, lập kế hoạch và đưa ra được cái plan
tối ưu nhất.

### 2. Narrow vs. Wide transformations

Transformations có hai loại và narrow và wide. Vậy hai loại này khác nhau như thế nào ?

![shuffle](image_1/check.png)

Sự khác biệt lớn nhất nằm ở việc có hay không có **shuffle**


- Với narrow transformation: map, filter --> Không có shuffle
- Với wide transformation: reduceByKey, sortByKey --> Có shuffle

**Vậy shuffle ở đây là gì ?**

Shuffle là hiện tượng xáo trộn dữ liệu giữa các node trong cluster để thu được kết quả đúng yêu cầu.

![narrow and wide](image_1/compare.png)

- Giải thích một chút về dữ liệu trong ảnh:
  + Bối cảnh: Giả sử có một file dữ liệu lớn với 50k record. Yêu cầu tìm số lượng order của mỗi loại status(Close, complete, pending.....)
  + Sử dụng hệ thống Spark, dữ liệu trong hdfs được chia thành 5 phần do 5 node trong cluster quản lí. Mỗi node quản lí 10k record.
  + Để giải quyết tìm số lượng order của các status, ta thực hiện quy trình như trong ảnh: Load to Memory --> Map --> reduceByKey --> Collect
  + Ở đây ta nhìn vào **map**: Mặc dù dữ liệu chuyển đổi từ string sang dictionary tuy nhiên bản chất nó vẫn vậy và nằm trong 1 node
  + Ở đây ta nhìn vào **ReduceByKey**: Để tổng hợp số lượng Order của mỗi status, bắt buộc phải đưa dữ liệu về một chỗ, ở đây cụ thể là một node để tính. Ví dụ như trên hình trên, ta đã di chuyển dữ liệu Close về lại node đầu tiên để tổng hợp và tính.
  + Việc đưa dữ liệu di chuyển rồi tổng hợp về một node như vậy gọi là Shuffle
- Chỉ có **reduceByKey** mới shuffle nên gọi nó là **Wide**, còn **map** bản chất dữ liệu vẫn nằm tại một node nên gọi là **narrow**
- Vì việc di chuyển dữ liệu giữa các node như vậy tốn rất nhiều dung lượng bộ nhớ làm cho chương trình thực thi bị chậm lại, nên trong các phép biến đổi transformation, ta ưu tiên đưa các narrow transformation lên trên đầu và đưa các wide transformation thực hiện ở các bước cuối để tối ưu hóa hiệu năng.

### 3. Các transformations phổ biến